In [1]:
import numpy as np

# An attempt to make finite element code to understand the needs with
# respect to continuum mechanics. An element has three coordinate states
# with reference, intial, and current states. The reference element is
# described below with the corresponding node numbering and reference
# coordinate system where the shape functions originate.
# 
#       n2-------n3  n0 = (-1,-1)    
#       |         |  n1 = ( 1,-1)
#       |         |  n2 = (-1, 1)
#       |         |  n3 = ( 1, 1)
#       n0-------n1
#
# This is all done in a 2D space. This is using four point gaussian 
# quadrature.

In [126]:
# vector and matrix functions (I don't want to use np.array because reasons)

# determinant of a matrix m   
def det(m):
    return m[0][0]*m[1][1]-m[0][1]*m[1][0]
   
# inverse of a matrix m
def inv(m):
    d = det(m)
    if(d == 0):
        raise NameError("matrix determinant is 0")
    return [[m[1][1]/d,-m[0][1]/d],
            [-m[1][0]/d,m[0][0]/d]]

# matrix times a vector
def mxv(m, v):
    return [m[0][0]*v[0]+m[0][1]*v[1],m[1][0]*v[0]+m[1][1]*v[1]]
    
# tensor product of vectors
def vov(u, v):
    return [[u[0]*v[0],u[0]*v[1]],[u[1]*v[0],u[1]*v[1]]]
    
# matrix addition
def maddm(m, n):
    return [[m[0][0]+n[0][0],m[0][1]+n[0][1]],[m[1][0]+n[1][0],m[1][1]+n[1][1]]]
    
# vector addition
def vaddv(u, v):
    return [u[0]+v[0],u[1]+v[1]]
    
# constant times vector
def cxv(c, v):
    return [c*v[0],c*v[1]]
    
# constant times matrix
def cxm(c, m):
    return [[c*m[0][0],c*m[0][1]],[c*m[1][0],c*m[1][1]]]
    
# matrix transpose
def mT(m):
    return [[m[0][0],m[1][0]],[m[0][1],m[1][1]]]

In [127]:
# define the node class
class node:
    # initalize
    def __init__(self, xc, yc):
        # 2D coordinates
        self.x = [xc, yc]

In [160]:
# define the 2D first order element class
class elem:
    # reference element quadrature point locations
    qr = [[-np.sqrt(1/3),-np.sqrt(1/3)],[np.sqrt(1/3),-np.sqrt(1/3)],
          [-np.sqrt(1/3),np.sqrt(1/3)],[np.sqrt(1/3),np.sqrt(1/3)]]
    # number of qudrature points
    n_qp = 4
    # number nodes (degrees of freedom)
    n_dof = 4
    
    # initialize
    def __init__(self, n0, n1, n2, n3):
        # quadrature point phi values
        self.phi_qp = []
        # quadrature point dphi_de values
        self.dphi_de_qp = []
        # qudrature point dX_de values
        self.dX_de_qp = []
        # point to current node locations
        self.n = [n0, n1, n2, n3]
        # initial node coordinates
        self.N = [node(n0.x[0], n0.x[1]), node(n1.x[0], n1.x[1]), 
                  node(n2.x[0], n2.x[1]), node(n3.x[0], n3.x[1])]
        # find initial quadrature point locations
        self.q = self.phi_QP()
        # find JxW for each quadrature point
        self.JxW_qp = self.JxW()
    
    # shape functions evaluated at a refernce coordinate x
    def phi(self, x):
        # error for improper reference coordinate query
        if((abs(x[0]) > 1) or (abs(x[1]) > 1)):
            raise NameError("reference coordinates must be in [-1,1]X[-1,1]")
        return [(1.0-x[0])*(1.0-x[1])/4.0,(1.0+x[0])*(1.0-x[1])/4.0,
                (1.0-x[0])*(1.0+x[1])/4.0,(1.0+x[0])*(1.0+x[1])/4.0];
        
    # position of a reference point x in the initial cooridinate system
    def phi_QP(self):
        QP = []
        for i in range(self.n_qp):
            p = self.phi(self.qr[i])
            self.phi_qp.append(p)
            X = [0.0, 0.0]
            for j in range(self.n_dof):
                X = vaddv(X,cxv(p[j],self.N[j].x))
            QP.append(X)
        return QP;

    # gradient of the shape functions evaluated at a refernce coordinate x
    def dphi_de(self, x):
        # error for improper reference coordinate query
        if((abs(x[0]) > 1) or (abs(x[1]) > 1)):
            raise NameError("reference coordinates must be in [-1,1]X[-1,1]")
        return [[(-1.0+x[1])/4.0,(-1.0+x[0])/4.0], 
                [ (1.0-x[1])/4.0,(-1.0-x[0])/4.0], 
                [(-1.0-x[1])/4.0, (1.0-x[0])/4.0], 
                [ (1.0+x[1])/4.0, (1.0+x[0])/4.0]]

    # return JxW at qp coordinates x
    def JxW(self):
        JxW = []
        for i in range(self.n_qp):
            jac = [[0,0],[0,0]]
            dp = self.dphi_de(self.qr[i])
            self.dphi_de_qp.append(dp)
            for j in range(self.n_dof):
                jac = maddm(jac, vov(self.N[j].x, dp[j]))
            self.dX_de_qp.append(jac)
            JxW.append(det(jac))
        return JxW
     
    # return FF
    def FF_qp(self):
        FF = []
        for i in range(self.n_qp):
            ff = [[0,0],[0,0]]
            dp = self.dphi_de_qp[i]
            nt = mT(inv(self.dX_de_qp[i]))
            for j in range(self.n_dof):
                ff = maddm(ff,vov(self.n[j].x, mxv(nt,dp[j])))
            FF.append(ff)
        return FF
            

In [167]:


# define nodes
n0 = node(0.0,0.0); n1 = node(1.0, 0.0)
n2 = node(0.0, 1.0); n3 = node(2.0, 1.0)

e = elem(n0, n1, n2, n3)

#n3.x = [0.0,1.0]
FF = e.FF_qp()
print(FF)



[[[1.0, 0.0], [0.0, 1.0]], [[1.0, 0.0], [0.0, 1.0]], [[1.0, 0.0], [0.0, 1.0]], [[1.0, 0.0], [0.0, 1.0]]]
